In [165]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import scipy.io as sio
from time import time
import pickle as pkl
import multiprocessing
import sys
from datetime import datetime,timedelta

def analyse(ret,trade_df_dict,hold_df_dict,initial_money,plotting=False):
    date_list = list(trade_df_dict.keys())
    divid_ret_df,trade_ratio_df = divid_earn_seq(date_list, hold_df_dict, trade_df_dict)
    print(trade_ratio_df)
    ###
    ret = ret + 0.0001
    nv = (ret+1).cumprod()
    nv_ls = (divid_ret_df['ls']+1).cumprod()
    nv_t0 = (divid_ret_df['t0']+1).cumprod()
    # nv_open = (divid_ret_df['openret']+1).cumprod()
    mean_ret = ret.mean() * 250
    mean_std = ret.std() * np.sqrt(250)
    sharp_ratio = mean_ret / mean_std
    dd_s = nv.cummax() - nv
    mean_dd = dd_s.mean()
    max_dd = dd_s.max()
    dd_date = dd_s.sort_values(ascending=False).index[0]

    # winrate_dict = {}
    # use_dict = {}
    # for date in ret.index:
    #     trade_df_raw = trade_df_dict[date]
    #     trade_df_raw['amt'] = trade_df_raw['volume'] * trade_df_raw['price']
    #     trade_df = -trade_df_raw.groupby('code')['amt'].sum()
    #     winrate_df = (trade_df > 0).astype(int)
    #     winrate = winrate_df.sum() / winrate_df.count()
    #     winrate_dict[date] = winrate
    #     use_ratio = trade_df_raw['amt'].abs().sum() / 2 / initial_money
    #     use_dict[date] = use_ratio
    # winrate = pd.Series(winrate_dict)
    # use_ratio = pd.Series(use_dict)
    
    if plotting is True:
        print('------绝对收益------')
        print('年化收益:' + str(mean_ret))
        print('多空收益:' + str(divid_ret_df['ls'].mean() * 250))
        print('t0收益:' + str(divid_ret_df['t0'].mean() * 250))
        # print('open收益:' + str(divid_ret_df['openret'].mean() * 250))
        print('年化波动:' + str(mean_std))
        print('夏普比率:' + str(sharp_ratio))
        print('平均回撤:' + str(mean_dd))
        print('最大回撤:' + str(max_dd))
        print('回撤日期:' + str(dd_date))
        # print('平均胜率:' + str(winrate.mean()))
        # print('平均底仓使用率:' + str(use_ratio.mean()))
        
        fig, ax1 = plt.subplots(figsize=(15, 7.5))
        ax1.plot(nv, label='strategy')
        ax1.plot(nv_ls, label='long_short')
        ax1.plot(nv_t0, label='t0')
        # ax1.plot(nv_open, label='open')
        ax1.set_xlabel('Date')
        ax1.set_ylabel('Net Value')
        ax1.legend()
        ax2 = ax1.twinx()
        ax2.bar(dd_s.index, dd_s, color='red', alpha=0.5)
        ax2.set_ylabel('Max Drawdown')
        plt.title('NV and Max Drawdown')
        plt.show()
        # winrate.plot(figsize=(15, 3),color='red')
        # #在50%胜率线上画一条横线虚线，颜色为蓝色
        # plt.axhline(0.5, color='b', linestyle='--')
        # plt.show()
        # #用红线标注平均胜率

    info_s = pd.Series([mean_ret, mean_std, sharp_ratio, mean_dd, max_dd, dd_date],
                       index=['年化收益', '年化波动', '夏普比率', '平均回撤', '最大回撤', '回撤日期'])
    return info_s

def get_secdata(date_str):
    sec_lobdata_path = r'/home/datamake8/hft_database/nas3'
    sec_bp1 = pd.read_feather(os.path.join(sec_lobdata_path,'sec_bp1_table',date_str+'.fea')).set_index('time')
    sec_bp1.columns = ['%06d'%int(x) for x in sec_bp1.columns]
    sec_sp1 = pd.read_feather(os.path.join(sec_lobdata_path,'sec_sp1_table',date_str+'.fea')).set_index('time')
    sec_sp1.columns = ['%06d'%int(x) for x in sec_sp1.columns]

    sec_open = pd.read_feather(os.path.join(r'/home/datamake8/hft_database/nas3/sec_fea_table','sec_open_table',date_str+'.fea')).set_index('second').fillna(method='ffill')
    sec_open.columns = ['%06d'%int(x) for x in sec_open.columns]
    return sec_bp1, sec_sp1, sec_open

def get_min_data(date_str):
    min_data = pd.read_feather(params.min_data_path + '//' + date_str + '.fea')
    min_data['StockID'] = min_data['StockID'].apply(lambda x:x[2:]).astype(str)
    min_data['time'] = min_data['time'].astype(int)
    #需要转换time的格式，与因子的对齐
    min_data['time'] = min_data['time'].apply(lambda x:x-40-1 if x%100==0 else x-1)
    amt_data = min_data[['time','StockID','amount']].pivot_table(index='time',columns='StockID',values='amount').sort_index(axis=0).sort_index(axis=1)
    vol_data = min_data[['time','StockID','vol']].pivot_table(index='time',columns='StockID',values='vol').sort_index(axis=0).sort_index(axis=1)
    vwap_data = amt_data.loc[930:1000].sum(axis=0) / (vol_data.loc[930:1000].sum(axis=0)+1e-7)
    return vwap_data

In [166]:
with open(r'/home/datamake8/data0/t0_backtest/T0多空策略0.636466.pkl','rb') as f:
    ans_dict = pkl.load(f)
hold_df_dict = ans_dict['hold_df_dict']
trade_df_dict = ans_dict['trade_df_dict']
tot_account_dict = ans_dict['tot_account_dict']
para = ans_dict['params']
ret_dict = ans_dict['ret_dict']
para

{'im': 100000000,
 'min': 0,
 'rank1': 2.5,
 'rank2': 3.5,
 'back': 2,
 'amp_ratio': 0.5,
 'score_ratio_1m': 0.2,
 'score_ratio_5m': 0.6}

In [128]:
date_list = list(tot_account_dict.keys())

In [156]:
close_price = pd.read_pickle('/home/datamake8/data0/ohlc_fea/CLOSE_PRICE.pkl').replace(0,np.nan).ffill()
close_price.index = pd.to_datetime(close_price.index).strftime('%Y%m%d')
daily_amount = pd.read_pickle('/home/datamake8/data0/ohlc_fea/TURNOVER_VALUE.pkl').fillna(0)
daily_amount.index = pd.to_datetime(daily_amount.index).strftime('%Y%m%d')
open_price = pd.read_pickle('/home/datamake8/data0/ohlc_fea/OPEN_PRICE.pkl').replace(0,np.nan).ffill().shift(-1)
open_price.index = pd.to_datetime(open_price.index).strftime('%Y%m%d')

total_ret_se = {}
ls_ret_se = {}
t0_ret_se = {}
open_ret_se = {}

# for date in tqdm(date_list):
date = date_list[60]

date_str = date.strftime('%Y%m%d')
open_df, close_df = hold_df_dict[date]
vwap_data = close_price.loc[date.strftime('%Y%m%d')].reindex(close_df.index)
b1_price, s1_price, sec_open = get_secdata(date_str)
close_cash, close_lock_cash = close_df['cash'].iloc[0], close_df['lock_cash'].iloc[0]



In [158]:
#读取交易记录，根据交易记录还原交易
trade_df = trade_df_dict[date]
trade_code_list = trade_df['code'].unique()
open_code_list = open_df.index.tolist()
all_code_list = sorted(list(set(trade_code_list) | set(open_code_list)))
account_df = open_df.reindex(index=all_code_list).fillna(0)
account_cash, account_lock_cash = account_df['cash'].iloc[0], account_df['lock_cash'].iloc[0]
for i in range(trade_df.shape[0]):
    trade_record = trade_df.iloc[i]
    rqflag = trade_record['rqflag']
    code = trade_record['code']
    volume = trade_record['volume']
    time = int(trade_record['time'].split(' ')[1])
    if volume > 0:
        if time < 144000 or time >= 145000:
            price = s1_price.loc[time,code]
        else:
            price = sec_open.loc[time,code]
    else:
        if time < 144000 or time >= 145000:
            price = b1_price.loc[time,code]
        else:
            price = sec_open.loc[time,code]
    
    if rqflag == 'cash':
        account_cash -= volume * price * {0:0.9992,1:1}[int(volume>=0)]
        account_df.loc[code,'hold_vol'] += volume
    else:
        account_lock_cash -= volume * price * {0:0.9992,1:1}[int(volume>=0)]
        account_df.loc[code,'debt_vol'] += volume

#计算收盘时的净值
account_df['hold_amt'] = account_df['hold_vol'] * vwap_data.reindex(account_df.index).fillna(0)
account_df['debt_amt'] = account_df['debt_vol'] * vwap_data.reindex(account_df.index).fillna(0)
account_df['rq_amt'] = account_df['rq_vol'] * vwap_data.reindex(account_df.index).fillna(0)
account_df['cash'] = account_cash
account_df['lock_cash'] = account_lock_cash

In [159]:
close_df

,hold_vol,hold_amt,debt_vol,debt_amt,rq_vol,rq_amt,cash,lock_cash
000001,0.0,0.0,0.0,0.0,0.0,0.0,2.760330e+07,7.183214e+07
000002,0.0,0.0,0.0,0.0,0.0,0.0,2.760330e+07,7.183214e+07
000006,0.0,0.0,0.0,0.0,0.0,0.0,2.760330e+07,7.183214e+07
000007,0.0,0.0,0.0,0.0,0.0,0.0,2.760330e+07,7.183214e+07
000008,0.0,0.0,0.0,0.0,0.0,0.0,2.760330e+07,7.183214e+07
...,...,...,...,...,...,...,...,...
688799,0.0,0.0,0.0,0.0,0.0,0.0,2.760330e+07,7.183214e+07
688800,0.0,0.0,0.0,0.0,0.0,0.0,2.760330e+07,7.183214e+07
688819,0.0,0.0,0.0,0.0,0.0,0.0,2.760330e+07,7.183214e+07
688981,0.0,0.0,0.0,0.0,0.0,0.0,2.760330e+07,7.183214e+07


In [160]:
account_df

,hold_vol,hold_amt,debt_vol,debt_amt,rq_vol,rq_amt,cash,lock_cash
000001,0.0,0.0,0.0,0.0,0.0,0.0,2.760330e+07,7.183214e+07
000002,0.0,0.0,0.0,0.0,0.0,0.0,2.760330e+07,7.183214e+07
000006,0.0,0.0,0.0,0.0,0.0,0.0,2.760330e+07,7.183214e+07
000007,0.0,0.0,0.0,0.0,0.0,0.0,2.760330e+07,7.183214e+07
000008,0.0,0.0,0.0,0.0,0.0,0.0,2.760330e+07,7.183214e+07
...,...,...,...,...,...,...,...,...
688799,0.0,0.0,0.0,0.0,0.0,0.0,2.760330e+07,7.183214e+07
688800,0.0,0.0,0.0,0.0,0.0,0.0,2.760330e+07,7.183214e+07
688819,0.0,0.0,0.0,0.0,0.0,0.0,2.760330e+07,7.183214e+07
688981,0.0,0.0,0.0,0.0,0.0,0.0,2.760330e+07,7.183214e+07


In [ ]:
#使用收盘价模拟交易，看多空持仓的收益是多少
open_value = open_df['hold_amt'].sum() + open_df['debt_amt'].sum() + open_cash + open_lock_cash - open_df['rq_amt'].sum()
hold_vol_diff = close_df['hold_vol'] - open_df['hold_vol']
rq_vol_diff = (close_df['rq_vol'] - close_df['debt_vol']) - (open_df['rq_vol'] - open_df['debt_vol'])
now_cash = open_cash
for code in hold_vol_diff.index:
    vol = hold_vol_diff[code]
    if vol==0: continue
    now_cash -= vol * vwap_data[code] * {0:1.0008,1:1}[int(vol>=0)]
now_lock_cash = open_lock_cash
for code in rq_vol_diff.index:
    vol = rq_vol_diff[code]
    if vol==0: continue
    now_lock_cash += vol * vwap_data[code] * {1:1.0008,0:1}[int(vol>=0)]

ls_earn = close_df['hold_amt'].sum() + close_df['debt_amt'].sum() + now_cash + now_lock_cash - close_df['rq_amt'].sum()
close_value = close_df['hold_amt'].sum() + close_df['debt_amt'].sum() + close_cash + close_lock_cash - close_df['rq_amt'].sum()
total_ret = (close_value - open_value) / open_value
ls_ret = (ls_earn - open_value) / open_value
t0_ret = total_ret - ls_ret
total_ret_se[date] = total_ret
ls_ret_se[date] = ls_ret
t0_ret_se[date] = t0_ret
#用第二天开盘价计算收益
pre_open_price = open_price.loc[open_price.index.tolist()[open_price.index.tolist().index(date.strftime('%Y%m%d'))-1]].reindex(open_df.index)
today_open_price = open_price.loc[date.strftime('%Y%m%d')].reindex(close_df.index)
pre_open_value = open_df['hold_vol'] @ pre_open_price + open_df['debt_vol'] @ pre_open_price + open_cash + open_lock_cash - open_df['rq_vol'] @ pre_open_price
next_open_value = close_df['hold_vol'] @ today_open_price + close_df['debt_vol'] @ today_open_price + now_cash + now_lock_cash - close_df['rq_vol'] @ today_open_price
next_open_ret = (next_open_value - pre_open_value) / pre_open_value
open_ret_se[date] = next_open_ret

#统计每日成交量占比
trade_ratio_dict = {}
for date in tqdm(date_list):
trade_df = trade_df_dict[date]
trade_df['amt'] = trade_df['amt'].abs()
trade_df = trade_df.groupby('code')['amt'].sum()
trade_df = trade_df / daily_amount.loc[date.strftime('%Y%m%d')].reindex(trade_df.index)
trade_ratio_dict[date] = trade_df.describe()